In [1]:
from google.cloud import bigquery
import pandas as pd
import numpy as np
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="BigQuery_Key.json"

In [2]:
client = bigquery.Client()

In [3]:
query = '''SELECT * FROM `as24-general-project.TEMPORARY_TABLES.Braulio_Berlanga_Source_Leads_DE_2`  order by date,sessionid,hit_number'''

In [4]:
query_rslt = client.query(query).to_dataframe()

In [5]:
df = query_rslt

In [23]:
df.head(5)

,date,device_category,visitorid,sessionid,hit_number,source_medium,lading_page,direct_traffic,page,recomended,...,offer_type,listing_id,emails,calls,wassp,total_enquiries,session_wth_lead,visitor_enquiry_order,enquiry_order_per_listing,sorted_calc
0,20220215,desktop,100015663729921831,1000156637299218311644915331,1,google / cpc,vp-de/all/all/home,True,vp-de/all/all/home,not recomended,...,None,None,0,0,0,0,1,NaN,NaN,no from search
1,20220215,desktop,100015663729921831,1000156637299218311644915331,2,google / cpc,vp-de/all/all/home,True,vp-de/all/all/home,not recomended,...,None,None,0,0,0,0,1,NaN,NaN,no from search
2,20220215,desktop,100015663729921831,1000156637299218311644915331,3,google / cpc,vp-de/all/all/home,True,vp-de/all/all/home,not recomended,...,None,None,0,0,0,0,1,NaN,NaN,no from search
3,20220215,desktop,100015663729921831,1000156637299218311644915331,4,google / cpc,vp-de/all/all/home,True,vp-de/all/all/home,not recomended,...,None,None,0,0,0,0,1,NaN,NaN,no from search
4,20220215,desktop,100015663729921831,1000156637299218311644915331,5,google / cpc,vp-de/all/all/home,True,vp-de/all/all/home,not recomended,...,None,None,0,0,0,0,1,NaN,NaN,no from search


In [25]:
## comparing with GA
df[['device_category','total_enquiries']].groupby('device_category').sum('total_enquiries')

,total_enquiries
device_category,
desktop,317559
mobile,624349
tablet,24350


In [26]:
## enquiries by sort type
df[['sorted','total_enquiries']].groupby('sorted').sum('total_enquiries')

,total_enquiries
sorted,
age|ascending,0
age|descending,9
createddate|descending,0
distance|ascending,0
leasing|ascending,0
mileage|ascending,1
mileage|descending,0
power|ascending,0
power|descending,0


In [27]:
a = len(df[(df['detail_list'].str.contains('list')==True) & (df['total_enquiries']>0)])
b = len(df[((df['detail_list'].str.contains('list')==False) | (df['detail_list'].isnull() == True)) & (df['total_enquiries']>0)])

print('total enquiries: {}, from search: {}, not from search: {}'.format(a+b,a,b))

total enquiries: 966258, from search: 341784, not from search: 624474


In [28]:
a = len(df[(df['detail_list'].str.contains('list')==True) & (df['total_enquiries']>0)])
b = len(df[((df['detail_list'].str.contains('list')==False) | (df['detail_list'].isnull() == True)) & (df['total_enquiries']>0)])
total = a+b

print('from search: {}%, not from search: {}%'.format(round((a/total)*100),round((b/total)*100)))

from search: 35%, not from search: 65%


### MAIN TRANSFORMATION

In [29]:
list_srt = df['sorted'].tolist()
list_pg = df['detail_list'].tolist()
list_enquiry_srt = []
n=0
for x,y in zip(list_srt,list_pg):
    try:
        if x == None:
            if 'listpage' in y:
                if list_enquiry_srt[n-1] == 'no from search':
                    value = 'from list, but sort not defined'
                else:
                    value = list_enquiry_srt[n-1]
            else:
                value = 'no from search'                
        else:
            value = x
    except:
        value = 'no from search' 
    list_enquiry_srt.append(value)


        
df['sorted_calc'] = pd.DataFrame({'lead_sort':list_enquiry_srt})         
    

In [30]:
df.head(5)

,date,device_category,visitorid,sessionid,hit_number,source_medium,lading_page,direct_traffic,page,recomended,...,offer_type,listing_id,emails,calls,wassp,total_enquiries,session_wth_lead,visitor_enquiry_order,enquiry_order_per_listing,sorted_calc
0,20220215,desktop,100015663729921831,1000156637299218311644915331,1,google / cpc,vp-de/all/all/home,True,vp-de/all/all/home,not recomended,...,None,None,0,0,0,0,1,NaN,NaN,no from search
1,20220215,desktop,100015663729921831,1000156637299218311644915331,2,google / cpc,vp-de/all/all/home,True,vp-de/all/all/home,not recomended,...,None,None,0,0,0,0,1,NaN,NaN,no from search
2,20220215,desktop,100015663729921831,1000156637299218311644915331,3,google / cpc,vp-de/all/all/home,True,vp-de/all/all/home,not recomended,...,None,None,0,0,0,0,1,NaN,NaN,no from search
3,20220215,desktop,100015663729921831,1000156637299218311644915331,4,google / cpc,vp-de/all/all/home,True,vp-de/all/all/home,not recomended,...,None,None,0,0,0,0,1,NaN,NaN,no from search
4,20220215,desktop,100015663729921831,1000156637299218311644915331,5,google / cpc,vp-de/all/all/home,True,vp-de/all/all/home,not recomended,...,None,None,0,0,0,0,1,NaN,NaN,no from search


### One enquiry per UserId, per Listing, per Date

In [31]:
df_by_uld = df[df['visitor_enquiry_order']==1].copy()

## From search or not from search

In [32]:
def from_search(x):
    try:
        if ('list' in x):
            value = 'yes'
        else:
            value = 'no'
    except:
        value = 'no'
    return value
        

df_by_uld['from search'] = df_by_uld['detail_list'].apply(from_search)
df_by_uld.head(5)

,date,device_category,visitorid,sessionid,hit_number,source_medium,lading_page,direct_traffic,page,recomended,...,listing_id,emails,calls,wassp,total_enquiries,session_wth_lead,visitor_enquiry_order,enquiry_order_per_listing,sorted_calc,from search
24,20220215,desktop,100015663729921831,1000156637299218311644915331,25,google / cpc,vp-de/all/all/home,True,vp-de/vm/uc/detail|email-success,not recomended,...,459e5e4f-a8d6-47d5-976e-addaec99f7eb,1,0,0,1,1,1.0,1.0,standard|ascending,yes
63,20220215,mobile,1001443043423543809,10014430434235438091644946291,12,google / cpc,vp-de/all/all/home,False,vp-de/vm/uc/detail|call,recommendation|homepage-engine-itembased,...,f9e3bb8e-fe0f-484f-addb-fd220e6e2441,0,1,0,1,1,1.0,2.0,no from search,no
78,20220215,mobile,1001443043423543809,10014430434235438091644946570,13,google / organic,vp-de/vm/opt/uc/mmc-regional-hub,False,vp-de/vm/uc/detail|call,not recomended,...,62112681-5aed-4f9f-ba2b-324cca746e4d,0,1,0,1,1,1.0,1.0,age|descending,yes
233,20220215,mobile,1001990071933099033,10019900719330990331644949752,133,google / cpc,vp-de/all/all/home,False,vp-de/vm/uc/detail|email-success,not recomended,...,5d1b3437-0244-4571-b5bb-e218d48dc4be,1,0,0,1,1,1.0,2.0,standard|ascending,yes
353,20220215,mobile,1002889373660171500,10028893736601715001644920498,69,web-native-share / (not set),vp-de/vm/uc/expired-vehicle-ad,False,vp-de/vm/uc/detail|email-success,not recomended,...,815f57bc-3afe-4c40-8f67-e1f8d84db87a,1,0,0,1,1,1.0,1.0,no from search,no


## from list or dealer profile

In [33]:
df_by_uld['listing_id'].fillna('from dealer',inplace=True)
df_by_uld['listing_id']

24          459e5e4f-a8d6-47d5-976e-addaec99f7eb
63          f9e3bb8e-fe0f-484f-addb-fd220e6e2441
78          62112681-5aed-4f9f-ba2b-324cca746e4d
233         5d1b3437-0244-4571-b5bb-e218d48dc4be
353         815f57bc-3afe-4c40-8f67-e1f8d84db87a
                            ...                 
38703400    5f791e3e-898f-431a-80b4-15e351daccc1
38703418    dac9f3bc-2007-4bc1-8787-c311d3c2f116
38703440                             from dealer
38703470    ea942ca2-f45c-4522-b7b7-a0cbc7928fac
38703502    9678f6ea-7cb6-40c1-9a90-5ac28c166fc3
Name: listing_id, Length: 803099, dtype: object

In [34]:
##df_by_uld[df_by_uld['listing_id'].isnull() == True]

def enquiries_clicks_at(x):
    if x == 'from dealer':
        value = 'from dealer'
    else:
        value = 'from list'
    return value

df_by_uld['enquiries_clicks_at'] = df_by_uld['listing_id'].apply(enquiries_clicks_at)

In [35]:
df_by_uld[['listing_id','total_enquiries','enquiries_clicks_at']]

,listing_id,total_enquiries,enquiries_clicks_at
24,459e5e4f-a8d6-47d5-976e-addaec99f7eb,1,from list
63,f9e3bb8e-fe0f-484f-addb-fd220e6e2441,1,from list
78,62112681-5aed-4f9f-ba2b-324cca746e4d,1,from list
233,5d1b3437-0244-4571-b5bb-e218d48dc4be,1,from list
353,815f57bc-3afe-4c40-8f67-e1f8d84db87a,1,from list
...,...,...,...
38703400,5f791e3e-898f-431a-80b4-15e351daccc1,1,from list
38703418,dac9f3bc-2007-4bc1-8787-c311d3c2f116,1,from list
38703440,from dealer,1,from dealer
38703470,ea942ca2-f45c-4522-b7b7-a0cbc7928fac,1,from list


In [36]:
##df[df['sort_type'] == 'from list, but sort not defined']

In [37]:
##df_excel = df[df['sessionid'] == '31126386716556637901647181236'].copy()
##df_excel.to_csv('not_defined_1.csv')

In [38]:
##df_by_uld.columns

In [39]:
df_by_uld.to_csv('source_lead_7_DE.csv')
print('data ready')

data ready


In [16]:
total = df_by_uld['total_enquiries'].sum()
df_from_srch = df_by_uld[['from search','total_enquiries']].groupby('from search',as_index=False).sum('total_enquiries').copy()
df_from_srch['% of leads'] = round((df_from_srch['total_enquiries']/total)*100)
df_from_srch

,from search,total_enquiries,% of leads
0,no,308736,63.0
1,yes,181383,37.0


In [111]:
total_leads = df_by_uld['total_enquiries'].sum()
print('total enquiries: {}'.format(total_leads))

total enquiries: 490119


In [17]:
df_sort_tpy = df_by_uld[['sorted_calc','total_enquiries']].groupby('sorted_calc').sum('total_enquiries').copy()
df_sort_tpy['% of enquiries'] = round((df_sort_tpy['total_enquiries']/total)*100)
df_sort_tpy

,total_enquiries,% of enquiries
sorted_calc,,
age|descending,16769,3.0
createddate|descending,595,0.0
distance|ascending,16,0.0
"from list, but sort not defined",26449,5.0
leasing|ascending,1,0.0
mileage|ascending,571,0.0
mileage|descending,44,0.0
no from search,308637,63.0
power|ascending,11,0.0
